# Emissions units with Pint

In this notebook we give some examples of how units are handled in SCMData and are built on top of the [Pint](https://github.com/hgrecco/pint) package.

In [2]:
import warnings
import traceback

import pint
from pint.errors import DimensionalityError

from scmdata.units import UnitConverter

AttributeError: module 'numpy' has no attribute 'ndarray'

## UnitConverter

The `UnitConverter` class handles all unit conversions for us. It is used as shown.

In [ ]:
uc = UnitConverter("GtC/yr", "Mt CO2 / yr")
uc.convert_from(1)

In [ ]:
uc.convert_to(1)

## Pint Unit Registry

The `unit_registry` which sits underneath all conversions can be accessed via `UnitConverter`'s `unit_registry` property. Nevertheless, you should have no need to access it directly.

In [ ]:
unit_registry = uc.unit_registry

Having accessed the `unit_registry`, all the units available in SCMData can be shown like so.

In [ ]:
dir(unit_registry)

## Using Pint Directly

For completeness, below we show how to use pint directly. Note that all of these operations are used by `UnitConverter` so the user shouldn't ever have to access pint in this way.

With the `unit_registry`, we can also create Pint variables/arrays which are unit aware.

In [ ]:
one_carbon = 1 * unit_registry("C")
one_carbon

In [ ]:
type(one_carbon)

In [ ]:
one_co2 = 1 * unit_registry.CO2
three_n2o = 3 * unit_registry.N2O

Pint quantities also print in an intuitive way.

In [ ]:
print(one_co2)
print(three_n2o)

We can convert them to base units or to each other.

In [ ]:
print(one_carbon.to_base_units())
print(one_co2.to('C'))
print(three_n2o.to('N'))

Operations are units aware.

In [ ]:
print(one_carbon + one_co2)
print(one_carbon * one_co2)
print((one_carbon * one_co2).to_base_units())
print(one_carbon / one_co2)
print((one_carbon / one_co2).to_base_units())

If we have compound units (e.g. emissions units which are [mass] * [substance] / [time]), we can convert any bit of the unit we want.

In [ ]:
eg1 = 1 * unit_registry("Mt") * unit_registry("C") / unit_registry("yr")
print(eg1)
eg2 = 5 * unit_registry("t") * unit_registry("CO2") / unit_registry("s")
print(eg2)

In [ ]:
print(eg1.to("Gt CO2 / day"))
print(eg2.to("Gt C / yr"))

## Contexts

With a context, we can use metric conversion definitions to do emissions conversions that would otherwise raise a `DimensionalityError`. For example, converting CO2 to N2O using AR4GWP100 (where 298 tCO2 = 1 tN2O).

In [ ]:
ar4gwp100uc = UnitConverter("N2O", "CO2", context="AR4GWP100")
ar4gwp100uc.convert_from(1)

In [ ]:
ar4gwp100uc = UnitConverter("N2O", "CH4", context="AR4GWP100")
ar4gwp100uc.convert_from(1)

We can see which contexts we have (which we can use for e.g. metric conversions).

In [ ]:
ar4gwp100uc.contexts

Such context dependent conversions can also be done directly with Pint.

In [ ]:
base = 1 * unit_registry("N2O")
with unit_registry.context('AR4GWP100'):
    print(one_carbon)
    print(one_carbon.to("CO2"))
    print(one_carbon.to('N') + three_n2o)  # I am not sure why you need to force the conversion of `a` first...

Without a context to tell us about metrics, if we try to do an invalid conversion, a `DimensionalityError` will be raised.

In [ ]:
try:
    ar4gwp100uc = UnitConverter("N2O", "CO2")
    ar4gwp100uc.convert_from(1)
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

In [ ]:
try:
    base.to("CO2")
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

If the context you use does not have the conversion you request, a warning will be raised. Any subsequent conversions will result in NaN's.

In [ ]:
# modify the way the warning appears to remove the path,
# thank you https://stackoverflow.com/a/26433913
def custom_formatting(message, category, filename, lineno, file=None, line=None):
    return "{}: {}\n".format(category.__name__, message)

warnings.formatwarning = custom_formatting

ucnan = UnitConverter("N2O", "Halon2402", context="SARGWP100")
ucnan.convert_from(1)